In [14]:
!pip install turicreate

     |████████████████████████████████| 92.0MB 44kB/s 
     |████████████████████████████████| 86.4MB 58kB/s 
     |████████████████████████████████| 3.4MB 45.1MB/s 
     |████████████████████████████████| 327kB 52.7MB/s 
     |████████████████████████████████| 450kB 38.4MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 3.8MB 50.9MB/s 
  Created wheel for prettytable: filename=prettytable-0.7.2-cp36-none-any.whl size=13702 sha256=a3d904557ce391ffac1fe16fb3d08aee1b2db2c9a532d506290bc46a7fd8b648
  Stored in directory: /root/.cache/pip/wheels/80/34/1c/3967380d9676d162cb59513bd9dc862d0584e045a162095606
  Created wheel for resampy: filename=resampy-0.2.1-cp36-none-any.whl size=320849 sha256=1a0adb534bcad50de197dca5a46b800ef575768402740e3374b26af425465f52
  Stored in directory: /root/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=c3

In [3]:
from turicreate import SFrame 

In [15]:
from google.colab import files

In [16]:
uploaded = files.upload()

Saving amazon_baby.zip to amazon_baby.zip


In [17]:
!unzip amazon_baby.zip

Archive:  amazon_baby.zip
   creating: amazon_baby.sframe/
  inflating: amazon_baby.sframe/m_bfaa91c17752f745.frame_idx  
   creating: __MACOSX/
   creating: __MACOSX/amazon_baby.sframe/
  inflating: __MACOSX/amazon_baby.sframe/._m_bfaa91c17752f745.frame_idx  
  inflating: amazon_baby.sframe/m_bfaa91c17752f745.0000  
  inflating: __MACOSX/amazon_baby.sframe/._m_bfaa91c17752f745.0000  
  inflating: amazon_baby.sframe/dir_archive.ini  
  inflating: __MACOSX/amazon_baby.sframe/._dir_archive.ini  
  inflating: amazon_baby.sframe/m_bfaa91c17752f745.sidx  
  inflating: __MACOSX/amazon_baby.sframe/._m_bfaa91c17752f745.sidx  
 extracting: amazon_baby.sframe/objects.bin  
  inflating: __MACOSX/amazon_baby.sframe/._objects.bin  
  inflating: __MACOSX/._amazon_baby.sframe  


In [18]:
import turicreate as tc

products = tc.SFrame('amazon_baby.sframe')

In [19]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

In [20]:
products.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


In [21]:
#Creating the word count vector
products['word_count']  = tc.text_analytics.count_words(products['review'])

In [22]:
#Exploring the most popular products
products['name'].show()

Materializing SArray

In [23]:
#Explore the Vulli Sophie
giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']

In [24]:
len(giraffe_reviews)

785

In [25]:
giraffe_reviews.show()

Materializing SFrame

Warning: Skipping column 'word_count'. Unable to show columns of type 'dictionary'; only [int, float, str] can be shown.

Further warnings of unsupported type will be suppressed.

In [27]:
#Defining which reviews have positive or negative sentiment
#Things that are 4 or 5 stars are Positive Everything else is negative
products['rating'].show()


Materializing SArray

In [28]:
#Define what a postiive and negative sentiment
#Ignore 3 star reviews
products = products[products['rating'] != 3]




In [29]:
#Binary for Positive Negatives
products['sentiment'] = products['rating'] >=4

In [30]:
#Training a sentiment classifier
train_data, test_data = products.random_split(.8, seed = 0)

In [31]:
sentiment_model = tc.logistic_classifier.create(train_data, 
                                                target = 'sentiment',
                                                features = ['word_count'],
                                                validation_set = test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.669978  | 5.113553     | 0.938650          | 0.916466            |

| 3         | 12       | 2.669978  | 5.701267     | 0.927417          | 0.901814            |

| 4         | 14       | 1.326072  | 6.603155     | 0.945200          | 0.918088            |

| 9         | 20       | 1.326072  | 9.917249     | 0.977654          | 0.917698            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

In [34]:
#Evaluating a classifier & the ROC Curve
sentiment_model.summary

<bound method Model.summary of Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 57357
Number of examples             : 133448
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 57356

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : lbfgs
Solver iterations              : 10
Solver status                  : Completed (Iteration limit reached).
Training time (sec)            : 6.7157

Settings
--------
Log-likelihood                 : 9590.1752

Highest Positive Coefficients
-----------------------------
word_count[arghhhhhh]          : 49.2272
word_count[joovys]             : 34.2942
word_count[screencons]         : 29.1217
word_count[punchers]           : 27.5292
word_count[angrily]            : 27.3339

Lowest Negative Coefficients
----------------------------

In [32]:
#sentiment_model.evaluate(test_data)

{'accuracy': 0.9176975738650012,
 'auc': 0.9258242975424673,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  1397 |
 |      1       |        0        |  1344 |
 |      0       |        0        |  3931 |
 |      1       |        1        | 26632 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.951057941255245,
 'log_loss': 0.33047871872209794,
 'precision': 0.9501587641371436,
 'recall': 0.9519588218472976,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+--------------------+--------------------+-------+------+
 | threshold |        fpr         |        tpr         |   p   |  n   |
 +-----------+--------------------+--------------------+-------+------+
 |    0.0 

In [33]:
#Applying the learned model to understand the sentiment for Giraffe
giraffe_reviews['predicted_sentiment'] = sentiment_model.predict(giraffe_reviews, output_type='probability')

giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending = False)

In [34]:
##Sort by best reviews based on predicted sentiments 
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending = False)

In [35]:
giraffe_reviews.head()

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,I'll be honest...I boughtthis toy because all the ...,4.0,"{'around': 1.0,'explore': 1.0, 'they': ...",1.0
Vulli Sophie the GiraffeTeether ...,As a mother of 16monthold twins; I bought ...,5.0,"{'will': 1.0, '15months':1.0, 'would': 2.0, ...",1.0
Vulli Sophie the GiraffeTeether ...,"Sophie, oh Sophie, yourtime has come. My ...",5.0,"{'11': 1.0, 'prisrob':1.0, '12': 1.0, 'who': ...",1.0
Vulli Sophie the GiraffeTeether ...,We got this littlegiraffe as a gift from a ...,5.0,"{'out': 1.0, 'would':1.0, 've': 1.0, 'enou ...",0.9999999999998377
Vulli Sophie the GiraffeTeether ...,"As every mom knows, youalways want to give your ...",5.0,"{'whether': 1.0,'neutral': 1.0, 'gend ...",0.9999999999998284
Vulli Sophie the GiraffeTeether ...,My Mom-in-Law boughtSophie for my son whe ...,5.0,"{'penny': 1.0, 'little':1.0, 'perfect': 1.0, ...",0.9999999999997958
Vulli Sophie the GiraffeTeether ...,"My 4 month old son isteething, and I've tried ...",4.0,"{'worth': 1.0, 'works':1.0, 'teether': 1.0, ...",0.9999999999994914
Vulli Sophie the GiraffeTeether ...,Let me just start off byaddressing the choking ...,5.0,"{'question': 1.0, 'must':1.0, 'overall': 1.0, ...",0.9999999999941254
Vulli Sophie the GiraffeTeether ...,I'm not sure why Sophieis such a hit with the ...,4.0,"{'makers': 1.0, 'or':1.0, 'take': 1.0, 'can': ...",0.999999999987423
Vulli Sophie the GiraffeTeether ...,"I admit, I didn't getSophie the Giraffe at ...",4.0,"{'dye': 1.0, 'of': 1.0,'cause': 1.0, 'fade': ...",0.9999999999829475


In [41]:
#Exploring the most positive and negative aspects of a product
giraffe_reviews[0]['review']

"As a mother of 16month old twins; I bought Sophie [1 for each, of course] when they were 4 months old after careful reading of all reviews. I heard great things about Sophie and wanted to give her a try. At 4 months babies can't do much more than grasp and semi gnaw on Sophie. For many months I had to squeeze Sophie myself [which I personally enjoyed] and set it on their laps. They LOVED Sophie. The squeak is LOUD and sounds exactly like a dog's squeaky chew toy, just for those who are wondering.As they grew and their motor skills developed to each milestone, Sophie gained more and more individual babytime. The twins were able to squeeze her themselves and chew on her around the clock. They love to throw her, stretch her, squeeze her, chew on her, drool on her... you name it, they have done it. One of the two Sophie's took an extended vacation out in the back yard [unbeknownst to me] and once found, a little water had her looking like a champ again... ready to face another day of play

In [42]:
giraffe_reviews[-1]['review']

"This children's toy is nostalgic and very cute. However, there is a distinct rubber smell and a very odd taste, yes I tried it, that my baby did not enjoy. Also, if it is soiled it is extremely difficult to clean as the rubber is a kind of porus material and does not clean well. The final thing is the squeaking device inside which stopped working after the first couple of days. I returned this item feeling I had overpaid for a toy that was defective and did not meet my expectations. Please do not be swayed by the cute packaging and hype surounding it as I was. One more thing, I was given a full refund from Amazon without any problem."

In [70]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [73]:
for i in selected_words:
    products[i] = products['word_count'].apply(lambda x: x.get(i,0))

In [74]:
products.head()

name,review,rating,word_count,sentiment,awesome
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1.0,'highly': 1.0, ...",1,0.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1.0, 'of': 1.0,'the': 1.0, 'than': 1.0, ...",1,0.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1.0, 'clever':1.0, 'approach': 2.0, ...",1,0.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ...",1,0.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",1,0.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ...",1,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",1,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",1,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",1,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1.0, 'now':1.0, 'its': 1.0, 'sti ...",1,0.0


In [82]:
df = tc.SFrame.to_dataframe(products)
for i in selected_words:
  print (i, df[i].sum())

awesome 3892.0
great 55791.0
fantastic 1664.0
amazing 2628.0
love 41994.0
horrible 1110
bad 4183
terrible 1146.0
awful 687
wow 425
hate 1107


In [83]:
train_data,test_data = products.random_split(.8, seed=0)

In [84]:
features=selected_words

In [85]:
selected_model = tc.logistic_classifier.create(train_data, 
                                                target = 'sentiment',
                                                features = features,
                                                validation_set = test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 0.728417     | 0.847708          | 0.846385            |

| 5         | 6        | 0.893064     | 0.847708          | 0.846385            |

| 6         | 7        | 1.053162     | 0.847708          | 0.846385            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [87]:
selected_model.summary

<bound method Model.summary of Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 12
Number of examples             : 133448
Number of classes              : 2
Number of feature columns      : 11
Number of unpacked features    : 11

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 1.1026

Settings
--------
Log-likelihood                 : 52926.808

Highest Positive Coefficients
-----------------------------
love                           : 1.3593
(intercept)                    : 1.3366
awesome                        : 1.1335
amazing                        : 1.1001
fantastic                      : 0.8858

Lowest Negative Coefficients
----------------------------
horrible     

In [90]:
sentiment_model = tc.logistic_classifier.create(train_data, 
                                                target = 'sentiment',
                                                features = ['word_count'],
                                                validation_set = test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.669978  | 4.271483     | 0.938650          | 0.916466            |

| 3         | 12       | 2.669978  | 4.887173     | 0.927417          | 0.901814            |

| 4         | 14       | 1.326072  | 5.844061     | 0.945200          | 0.918088            |

| 9         | 20       | 1.326072  | 9.241857     | 0.977654          | 0.917698            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

In [92]:
test_data['sentiment'].show()

Materializing SArray

In [93]:
baby = products[products['name'] == 'Baby Trend Diaper Champ']

In [105]:
baby['predicted_sentiment'] = sentiment_model.predict(baby, output_type='probability')


baby['predicted_sentiment2'] = selected_model.predict(baby, output_type='probability')

baby = baby.sort('predicted_sentiment', ascending = False)

In [106]:
baby.head()

name,review,rating,word_count,sentiment,awesome
Baby Trend Diaper Champ,I read a review belowthat can explain exactly ...,4.0,"{'key': 1.0, 'have': 1.0,'pieces': 1.0, 'betwe ...",1,0.0
Baby Trend Diaper Champ,I have never written areview for Amazon but I ...,5.0,"{'priceless': 1.0,'knows': 1.0, 'parent': ...",1,0.0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'price': 1.0,'suggestions': 1.0, ...",1,0.0
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5.0,"{'around': 1.0, 'any':1.0, 't': 1.0, 'isn': ...",1,0.0
Baby Trend Diaper Champ,Diaper Champ or DiaperGenie? That was my ...,5.0,"{'either': 1.0, 'be':1.0, 't': 1.0, 'not': ...",1,0.0
Baby Trend Diaper Champ,I am one of those super-critical shoppers who ...,5.0,"{'hope': 1.0, 'make':1.0, 'slower': 1.0, ...",1,0.0
Baby Trend Diaper Champ,I LOOOVE this diaperpail! Its the easies ...,5.0,"{'buy': 1.0, 'product':1.0, 'recommend': 1.0, ...",1,0.0
Baby Trend Diaper Champ,"As a first time mother, Iwanted to get the best ...",5.0,"{'ll': 1.0, 'baby': 1.0,'recommended': 1.0, ' ...",1,0.0
Baby Trend Diaper Champ,I see that there arecomplaints of stinkiness ...,5.0,"{'very': 1.0, 'told':1.0, 'all': 1.0, ...",1,0.0
Baby Trend Diaper Champ,I have a 10 year olddaughter and an 8 month ...,5.0,"{'sorry': 1.0, 'be': 1.0,'you': 2.0, 'sell': 1.0, ...",1,0.0


In [107]:
baby[0]['review']

"I read a review below that can explain exactly what we experienced. We've had it for 16 months and it has worked wonderful for us. No smells, change it out once a week, easy to clean. Then a diaper snagged this foam material in the head part, so I pulled the rest of the foam out. Big mistake!!! Now it can no loner retain the stinkiness and we're looking for a replacement. Be careful of overloading and never take out that foam piece that is cushioned between pieces. I have figured out that it is key to keeping the stink out."